In [20]:
import pandas as pd
import json

In [27]:
with open("../out/data.json") as fout:
    raw_data = json.load(fout)

question = []
response = []
for idx_thread, thread in enumerate(raw_data):
    for idx_post in range(int(len(thread) / 2)):
        next_posts = thread[idx_post:idx_post+2]
        if next_posts[0]["role"] == "user" and next_posts[1]["role"] == "system":
            question.append(next_posts[0]["text"])
            response.append(next_posts[1]["text"])
    if idx_thread == 200:
        break

# create dataframe
q_a_dataframe = pd.DataFrame(zip(question, response), columns=["instruction", "response"])
q_a_dataframe

,instruction,response
0,"Hi, I have a very basic notebook running in ou...",Cross-referenced post: galaxyproject/admins - ...
1,Any ideas yet what is going on with put and ge...,\n\n\n maikenp:\n\njupyter\n\n\nHi @maikenp\nS...
2,The key point being I think the use of the qua...,Ok! Progress Did you set galaxy_infrastructur...
3,@hxr do you have some advice on what would be ...,"Hi @maikenp sorry for the delay, too many thin..."
4,"Hi, I met an error when I used the Trinity too...",Hi @luweidong\nTry transcript assembly for a s...
...,...,...
351,Hi @wm75\nI did exactly what you told but i am...,"I’m sorry, but I don’t understand what the iss..."
352,I have a problem when trying to get genomic DN...,Hi @Mattiaf!\nI noticed a problem myself with ...
353,Thanks Jennifer.\nI get the “Chromosome by nam...,Thanks for info. Am reviewing with these extra...
354,Hello there!\nI am working with WGBS data and ...,"Hi @srinivasa_abishek,\nI’m trying to reproduc..."


In [28]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:\n"""

q_a_dataframe['prompt'] = q_a_dataframe["instruction"].apply(lambda x: template.format(x))
q_a_dataframe

,instruction,response,prompt
0,"Hi, I have a very basic notebook running in ou...",Cross-referenced post: galaxyproject/admins - ...,Below is an instruction that describes a task....
1,Any ideas yet what is going on with put and ge...,\n\n\n maikenp:\n\njupyter\n\n\nHi @maikenp\nS...,Below is an instruction that describes a task....
2,The key point being I think the use of the qua...,Ok! Progress Did you set galaxy_infrastructur...,Below is an instruction that describes a task....
3,@hxr do you have some advice on what would be ...,"Hi @maikenp sorry for the delay, too many thin...",Below is an instruction that describes a task....
4,"Hi, I met an error when I used the Trinity too...",Hi @luweidong\nTry transcript assembly for a s...,Below is an instruction that describes a task....
...,...,...,...
351,Hi @wm75\nI did exactly what you told but i am...,"I’m sorry, but I don’t understand what the iss...",Below is an instruction that describes a task....
352,I have a problem when trying to get genomic DN...,Hi @Mattiaf!\nI noticed a problem myself with ...,Below is an instruction that describes a task....
353,Thanks Jennifer.\nI get the “Chromosome by nam...,Thanks for info. Am reviewing with these extra...,Below is an instruction that describes a task....
354,Hello there!\nI am working with WGBS data and ...,"Hi @srinivasa_abishek,\nI’m trying to reproduc...",Below is an instruction that describes a task....


In [29]:
q_a_dataframe = q_a_dataframe[['prompt', 'response']]
q_a_dataframe

,prompt,response
0,Below is an instruction that describes a task....,Cross-referenced post: galaxyproject/admins - ...
1,Below is an instruction that describes a task....,\n\n\n maikenp:\n\njupyter\n\n\nHi @maikenp\nS...
2,Below is an instruction that describes a task....,Ok! Progress Did you set galaxy_infrastructur...
3,Below is an instruction that describes a task....,"Hi @maikenp sorry for the delay, too many thin..."
4,Below is an instruction that describes a task....,Hi @luweidong\nTry transcript assembly for a s...
...,...,...
351,Below is an instruction that describes a task....,"I’m sorry, but I don’t understand what the iss..."
352,Below is an instruction that describes a task....,Hi @Mattiaf!\nI noticed a problem myself with ...
353,Below is an instruction that describes a task....,Thanks for info. Am reviewing with these extra...
354,Below is an instruction that describes a task....,"Hi @srinivasa_abishek,\nI’m trying to reproduc..."


In [30]:
q_a_dataframe['text'] = q_a_dataframe["prompt"] + q_a_dataframe["response"]
q_a_dataframe.drop(columns=['prompt', 'response'], inplace=True)
display(q_a_dataframe), q_a_dataframe.shape

,text
0,Below is an instruction that describes a task....
1,Below is an instruction that describes a task....
2,Below is an instruction that describes a task....
3,Below is an instruction that describes a task....
4,Below is an instruction that describes a task....
...,...
351,Below is an instruction that describes a task....
352,Below is an instruction that describes a task....
353,Below is an instruction that describes a task....
354,Below is an instruction that describes a task....


(None, (356, 1))

In [31]:
from datasets import load_dataset
from datasets import Dataset
dataset = Dataset.from_pandas(q_a_dataframe).train_test_split(test_size=0.05, seed=42)

/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
import torch
from transformers import BioGptTokenizer, BioGptForCausalLM, set_seed
import re
import sys

model_path = "microsoft/biogpt"

tokenizer = BioGptTokenizer.from_pretrained(model_path)
model = BioGptForCausalLM.from_pretrained(model_path, torch_dtype=torch.float32)

model_modules = str(model.modules)
print(model_modules)

<bound method Module.modules of BioGptForCausalLM(
  (biogpt): BioGptModel(
    (embed_tokens): Embedding(42384, 1024, padding_idx=1)
    (embed_positions): BioGptLearnedPositionalEmbedding(1026, 1024)
    (layers): ModuleList(
      (0-23): 24 x BioGptDecoderLayer(
        (self_attn): BioGptAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layer_norm):

In [33]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import torch
from transformers.trainer_callback import TrainerCallback
import os
import sys
from transformers import BitsAndBytesConfig
from trl import SFTTrainer

target_modules = ['q_proj','v_proj']

lora_config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    target_modules = target_modules,
    task_type="CAUSAL_LM",
)

base_dir = "biogpt"

per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = 'adamw_hf'
learning_rate = 1e-5
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "linear"


from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir=base_dir,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    num_train_epochs = 5.0,
    logging_strategy="epoch",
    logging_steps=100,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)
    
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainer = SFTTrainer(
    model,
    train_dataset=dataset['train'],
    eval_dataset = dataset['test'],
    dataset_text_field="text",
    max_seq_length=256,
    args=training_args,
)

trainer.train()

trainable params: 786,432 || all params: 347,549,696 || trainable%: 0.22627900672944337


Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 154.54 examples/s]
/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,5.415300,5.041604
1,5.409200,5.028563
2,5.391500,5.018176
4,5.140400,5.011297
4,5.305500,5.009208


TrainOutput(global_step=105, training_loss=5.3308263142903645, metrics={'train_runtime': 441.5929, 'train_samples_per_second': 3.827, 'train_steps_per_second': 0.238, 'total_flos': 644394528595968.0, 'train_loss': 5.3308263142903645, 'epoch': 4.94})

In [37]:
# test fine-tuned model
# Number 1000, 1001 from data.json
test_strings = ["Hello!\nI am running a local instance of Galaxy (build 22.05). I installed the latest version of Deepvariant (1.4.0+galaxy0) which installed without any errors. However, when I try to run Deepvariant on BAM files output from HISAT2, the error “Fatal error: Exit code 127 ()” comes up. Further, it says that the tool generated the following error: “line 9: run_deepvariant: command not found”.\nWhen I look at the backend to see what process Galaxy is going through, even after installation of the tool, the following line keeps repeating on the command line interface:\nuvicorn.access INFO 2022-12-22 13:43:26,629 [pN:main.1,p:100965,tN:MainThread] 127.0.0.1:58158 - “GET /api/tool_shed_repositories?name=deepvariant&owner=iuc HTTP/1.1” 200\nI don’t understand this error. Could someone please help me out?\nI am running the same job on Galaxy.eu server and it is running (for a few hours now) but in the local instance in errors out pretty much instantly.\nThanks!",
"Dear Sir,\nKindly help in this regards I was trying to make a de novo contig using trinity and it is running since from one week.\nIs it ok??? or did I out something wrong\nKindly help",]

#response_test_strings = [
#"Hello, I can’t give a full answer but I can maybe guide you in the right direction and maybe someone that can give a better answer will reply.\nGiven the error it looks like deepvariant is not installed (not found). The tool is using a “docker tool dependency”, in other words it needs a container where deepvariant is installed. If you have not checked this yet then I think this is the place to start. Below two links where you may find some more information.\nhttps://docs.galaxyproject.org/en/master/admin/special_topics/mulled_containers.html\n  \n      \n\n      training.galaxyproject.org\n  \n\n  \n    \n\nGalaxy Training: Tool Dependencies and Containers\n\n  Galaxy is an open-source project. Everyone can contribute...\n\n\n  \n\n  \n    \n    \n  \n\n  \n\n\nThe requirement can be seen here:\n  \n\n      github.com\n  \n\n  \n    galaxyproject/tools-iuc/blob/master/tools/deepvariant/macros.xml\n\n\n      <macros>\n    <token name="@TOOL_VERSION@">1.4.0</token>\n    <token name="@SUFFIX_VERSION@">0</token>\n    <xml name="edam_ontology">\n        <edam_topics>                                                                                  \n            <edam_topic>topic_0199</edam_topic>\n        </edam_topics>\n        <edam_operations>\n            <edam_operation>operation_3227</edam_operation>\n        </edam_operations>\n    </xml>\n    <xml name="requirements">\n        <requirements>\n            <container type="docker">google/deepvariant:@TOOL_VERSION@</container>\n        </requirements>\n    </xml>\n    <xml name="citations">\n        <citations>\n            <citation type="doi">10.1038/nbt.4235</citation>\n        </citations>\n\n\n\n\n  This file has been truncated. show original\n\n  \n\n  \n    \n    \n  \n\n  \n\n", 
# "Hello @Sachin_Srivastava\nIf the job is running (yellow/peach dataset), it is usually best to allow it to run. The same is true for queued jobs (grey dataset). This applies to jobs (any tool) executed at a public Galaxy server.\n20 GB of fastq data – uncompressed – creates a very large assembly job. If it fails later on for exceeding resources (red dataset), you’ll need to do one or more of these:\n\nTry a rerun to eliminate cluster issues\nMore QA/QC on the input reads (always recommended)\nConsider downsampling the reads (tool: Seqtk)\nPossibly need to move to your own Galaxy server where more resources can be allocated. The GVL version of Cloudman is one option: https://launch.usegalaxy.org/catalog\n\n\nI added some tags to your post that will find prior Q&A about the above actions. Or, you can search the forum with those keywords (not all posts get tagged).\nYou didn’t state where you are working. But, if by chance at Galaxy Main https://usegalaxy.org, I can let you know that the cluster that runs Trinity (and Unicycler + RNA-Star) is very busy. Longer queue times are expected. If you delete the current job and rerun, that will only place your job back at the end of the queue again, extending wait time.\nThanks!"]


predictions = []
for test in test_strings:
  prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
  {}

  ### Response:""".format(test)
    
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda')

  generation_output = model.generate(
      input_ids=input_ids, max_new_tokens=156
  )
  predictions.append(tokenizer.decode(generation_output[0]))


def extract_response_text(input_string):
    start_marker = '### Response:'
    end_marker = '###'
    
    start_index = input_string.find(start_marker)
    if start_index == -1:
        return None
    
    start_index += len(start_marker)
    
    end_index = input_string.find(end_marker, start_index)
    if end_index == -1:
        return input_string[start_index:]
    
    return input_string[start_index:end_index].strip()

for i in range(3): 
  pred = predictions[i]
  text = test_strings[i]
  print(text+'\n')
  print("response: \n")
  print(pred)
  #print(extract_response_text(pred))
  print('--------')

Hello!
I am running a local instance of Galaxy (build 22.05). I installed the latest version of Deepvariant (1.4.0+galaxy0) which installed without any errors. However, when I try to run Deepvariant on BAM files output from HISAT2, the error “Fatal error: Exit code 127 ()” comes up. Further, it says that the tool generated the following error: “line 9: run_deepvariant: command not found”.
When I look at the backend to see what process Galaxy is going through, even after installation of the tool, the following line keeps repeating on the command line interface:
uvicorn.access INFO 2022-12-22 13:43:26,629 [pN:main.1,p:100965,tN:MainThread] 127.0.0.1:58158 - “GET /api/tool_shed_repositories?name=deepvariant&owner=iuc HTTP/1.1” 200
I don’t understand this error. Could someone please help me out?
I am running the same job on Galaxy.eu server and it is running (for a few hours now) but in the local instance in errors out pretty much instantly.
Thanks!

response: 

</s>Below is an instruction

IndexError: list index out of range